# Word2Vec Corpus

Reference: https://nlp.stanford.edu/projects/glove/

## Import Libraries

In [1]:
import numpy as np
import pandas as pd

from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

from utils.preprocess import getWord2VecCorpus
from utils.utils import getVecForm
from utils.models import simpleNN, simpleLSTM

import tensorflow as tf

In [2]:
PATH = './Datasets/'
DIMS = 300
PREPROCESS = 'word2vec'
MAXVECLEN = 30

## Read GloVe Data

In [3]:
VEC = getWord2VecCorpus()
VEC

## Import Dataset

In [4]:
df = pd.read_csv(PATH+'Sarcasm_Headlines_Detection.csv').dropna().reset_index(drop=True)
df

,headline,is_sarcastic
0,versace store clerk sue secret black code mino...,0
1,roseanne revival catch thorny political mood w...,0
2,mom start fear son web series close thing gran...,1
3,boehner want wife listen come alternative debt...,1
4,rowling wish snape happy birthday magical way,0
...,...,...
28608,tyson hold contest let fan submit new idea tor...,1
28609,increasingly cocky bernie sander announce will...,1
28610,cash strap zuckerberg force sell million faceb...,1
28611,grocery store bar actually great little happy ...,1


### Remove excessively long texts

In [5]:
tooLong = []
for i in range(len(df['headline'])):
    if len(df['headline'][i].split()) > MAXVECLEN:
        tooLong.append(i)
for i in tooLong:
    df = df.drop(i, axis=0).reset_index(drop=True)
df

,headline,is_sarcastic
0,versace store clerk sue secret black code mino...,0
1,roseanne revival catch thorny political mood w...,0
2,mom start fear son web series close thing gran...,1
3,boehner want wife listen come alternative debt...,1
4,rowling wish snape happy birthday magical way,0
...,...,...
28607,tyson hold contest let fan submit new idea tor...,1
28608,increasingly cocky bernie sander announce will...,1
28609,cash strap zuckerberg force sell million faceb...,1
28610,grocery store bar actually great little happy ...,1


In [6]:
X_, Y_ = df['headline'], df['is_sarcastic']

## Simple NN

### Convert dataset into Word Vector

In [7]:
X, Y = getVecForm(
    X = X_,
    Y = Y_,
    vec = VEC,
    dims=DIMS,
    preprocess=PREPROCESS,
    vectype='sum',
)

In [8]:
X.shape

(28612, 300)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, Y)

### Training a Simple NN

In [10]:
snn = simpleNN(X)
snn.fit(x_train, y_train, validation_split=0.3, epochs=5)

Epoch 1/5
470/470 [==============================] - 2s 3ms/step - loss: 0.5948 - accuracy: 0.6979 - val_loss: 0.5244 - val_accuracy: 0.7436
Epoch 2/5
470/470 [==============================] - 1s 3ms/step - loss: 0.5049 - accuracy: 0.7515 - val_loss: 0.5109 - val_accuracy: 0.7515
Epoch 3/5
470/470 [==============================] - 1s 3ms/step - loss: 0.4696 - accuracy: 0.7772 - val_loss: 0.5017 - val_accuracy: 0.7560
Epoch 4/5
470/470 [==============================] - 1s 3ms/step - loss: 0.4301 - accuracy: 0.8001 - val_loss: 0.5046 - val_accuracy: 0.7578
Epoch 5/5
470/470 [==============================] - 1s 3ms/step - loss: 0.3907 - accuracy: 0.8234 - val_loss: 0.5078 - val_accuracy: 0.7608


In [11]:
snn.evaluate(x_test, y_test)

224/224 [==============================] - 0s 1ms/step - loss: 0.5239 - accuracy: 0.7477


[0.5238915681838989, 0.7476583123207092]

## Simple LSTM (sum)

### Convert dataset into Word Vector

In [12]:
X, Y = getVecForm(
    X = X_,
    Y = Y_,
    vec = VEC,
    dims=DIMS,
    preprocess=PREPROCESS,
    vectype='sum',
    reshaping=(X.shape[0], 1, X.shape[1])
)

In [13]:
X.shape

(28612, 1, 300)

In [14]:
x_train, x_test, y_train, y_test = train_test_split(X, Y)

### Training a Simple LSTM

In [15]:
lstms = simpleLSTM(X)
lstms.fit(x_train, y_train, validation_split=0.3, epochs=10)

Epoch 1/10
470/470 [==============================] - 4s 4ms/step - loss: 0.6154 - accuracy: 0.6663 - val_loss: 0.5801 - val_accuracy: 0.7032
Epoch 2/10
470/470 [==============================] - 2s 3ms/step - loss: 0.5520 - accuracy: 0.7239 - val_loss: 0.5534 - val_accuracy: 0.7206
Epoch 3/10
470/470 [==============================] - 2s 3ms/step - loss: 0.5268 - accuracy: 0.7417 - val_loss: 0.5434 - val_accuracy: 0.7285
Epoch 4/10
470/470 [==============================] - 2s 3ms/step - loss: 0.5119 - accuracy: 0.7521 - val_loss: 0.5343 - val_accuracy: 0.7359
Epoch 5/10
470/470 [==============================] - 2s 3ms/step - loss: 0.4993 - accuracy: 0.7616 - val_loss: 0.5250 - val_accuracy: 0.7366
Epoch 6/10
470/470 [==============================] - 2s 3ms/step - loss: 0.4886 - accuracy: 0.7669 - val_loss: 0.5198 - val_accuracy: 0.7392
Epoch 7/10
470/470 [==============================] - 2s 3ms/step - loss: 0.4795 - accuracy: 0.7746 - val_loss: 0.5184 - val_accuracy: 0.7418
Epoch 

In [16]:
lstms.evaluate(x_test, y_test)

224/224 [==============================] - 0s 1ms/step - loss: 0.5011 - accuracy: 0.7560


[0.5010948777198792, 0.7560464143753052]

## Simple LSTM (vector)

### Convert dataset into Word Vector

In [17]:
X, Y = getVecForm(
    X = X_,
    Y = Y_,
    vec = VEC,
    dims=DIMS,
    preprocess=PREPROCESS,
    vectype='vector',
    MaxvecLen=MAXVECLEN
)

In [18]:
X.shape

(28612, 30, 300)

In [19]:
x_train, x_test, y_train, y_test = train_test_split(X, Y)

### Training a Simple LSTM

In [20]:
lstmv = simpleLSTM(X)
lstmv.fit(x_train, y_train, validation_split=0.3, epochs=10)

Epoch 1/10
470/470 [==============================] - 4s 7ms/step - loss: 0.6197 - accuracy: 0.6617 - val_loss: 0.5758 - val_accuracy: 0.7164
Epoch 2/10
470/470 [==============================] - 3s 6ms/step - loss: 0.5455 - accuracy: 0.7404 - val_loss: 0.5420 - val_accuracy: 0.7383
Epoch 3/10
470/470 [==============================] - 3s 6ms/step - loss: 0.5212 - accuracy: 0.7579 - val_loss: 0.5291 - val_accuracy: 0.7451
Epoch 4/10
470/470 [==============================] - 3s 6ms/step - loss: 0.5048 - accuracy: 0.7673 - val_loss: 0.5168 - val_accuracy: 0.7538
Epoch 5/10
470/470 [==============================] - 3s 6ms/step - loss: 0.4894 - accuracy: 0.7770 - val_loss: 0.5085 - val_accuracy: 0.7600
Epoch 6/10
470/470 [==============================] - 3s 6ms/step - loss: 0.4769 - accuracy: 0.7863 - val_loss: 0.4983 - val_accuracy: 0.7651
Epoch 7/10
470/470 [==============================] - 3s 6ms/step - loss: 0.4608 - accuracy: 0.7949 - val_loss: 0.4929 - val_accuracy: 0.7695
Epoch 

In [21]:
lstmv.evaluate(x_test, y_test)

224/224 [==============================] - 1s 3ms/step - loss: 0.4732 - accuracy: 0.7808


[0.4732193350791931, 0.7807912826538086]